## Compared to v1, v2 also calculates the 95% two-sided CI for rho, by basic bootstrap method with 100 iterations (can be very slow, as the MLE optimisation is repeated 100 times) 

In [4]:
import itertools as itt
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import os
import pandas as pd
import pickle
import scipy.optimize as spo
import scipy.special as spsp
import scipy.stats as sps
import seaborn as sns
import time

from matplotlib.ticker import FixedLocator

mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['legend.frameon'] = True
mpl.rcParams['legend.framealpha'] = 0.5
mpl.rcParams['legend.fontsize'] = 'large'

path_proj = os.getcwd()+ '/'
path_sc = path_proj + 'csv/Single_cell/'
path_sort = path_proj + 'csv/Sort/'
path_plot = path_proj + 'figures/'
print(path_proj)

#LOAD THE PROCESSED DATA
df = pd.read_csv(path_sc+'Pooled_data.csv', sep=';', decimal=',')

#GLOBAL VARIABLES FOR ITERATION
conds = ['P', 'P+ILs']
or_cells = ['SLAM-HSC', 'ST-HSC', 'MPP']

#SET A RANDOM SEED TO REPLICATE THE BOOTSTRAP CONFIDENCE INTEVALS
#BASED ON RANDOM SAMPLING WITH REPLACEMENT
np.random.seed(123)

/home/giulio/Desktop/Generational_multiplex_analysis/


In [5]:
#IMPORTANT: SAMPLING PROBABILITY MUST BE SPECIFIED FROM THE USER! 
r_sample = 0.71

In [6]:
#NOTE: A FULL FAMILY VECTOR IS THE VECTOR OF CELL COUNTS PER GENERATION (ONE GENERATION PER VECTOR ENTRY);
#A SAMPLED FAMILY VECTOR IS A FULL FAMILY VECTOR WHERE CELLS WERE NOT NECESSARILY COUNTED/OBSERVED

#BELOW ARE THE FUNCTIONS AND CLASSES TO DEFINE FULL AND SAMPLED FAMILY VECTORS

#GIVEN A SAMPLED FAMILY VECTOR, POPULATE THE NEXT GENERATION WITH ALL POSSIBLE COUNTS THAT ARE PHYSICALLY POSSIBLE
#(HERE IT IS ASSUMED THAT EACH DIVIDING CELLS PRODUCES TWO CELLS IN THE NEXT GENERATION)
def add_generation(family):
    i = len(family)
    past = [family[j] * 2**(i-j) for j in range(i)]
    max_val = 2**i - sum(past)
    new_fam_list = [family[:]+[k] for k in range(max_val+1)]
    return new_fam_list

#CREATE ALL SAMPLED FAMILY VECTORS UP TO GENERATION DD
def fam_generator(DD):
    possible_fam = [[0],[1]]   
    for DD in range(DD):
        temp = [add_generation(elem) for elem in possible_fam]
        possible_fam = list(itt.chain.from_iterable(temp))
    return possible_fam
    
#GIVEN A FULL FAMILY VECTOR, RETURNS ALL SAMPLED FAMILY VECTORS THAT MAY POSSIBLY DESCENT FROM IT  
def sub_trees_gen(ft):
    res_lst = [[float(k)] for k in range(int(ft[0])+1)]
    for max_n in ft[1:]:
        new_lst = [vec[:]+[float(k)] for k in range(int(max_n)+1)
                                     for vec in res_lst]
        res_lst = new_lst
    return [Sub_tree(np.asarray(vec), sum(np.subtract(ft,vec)))
                for vec in res_lst if sum(vec)!=0.]

#CLASS FOR SAMPLED FAMILY VECTORS
class Sub_tree(object):
    def __init__(self, st, lost_c):
        self.st = st
        self.lost_c = lost_c
        self.range = np.arange(len(st))[st>0].max() - np.arange(len(st))[st>0].min()
    def show_st(self):
        print( 'ST:', self.st, 'Lc:', self.lost_c)

#CLASS FOR FULL FAMILY VECTORS
class Full_tree(object):
    def __init__(self, ft):
        self.ft = ft
        self.ebc = cells_arrived_in_gen(ft)
        self.st_lst = sub_trees_gen(ft)
        self.range = np.arange(len(ft))[ft>0].max() - np.arange(len(ft))[ft>0].min()
    def show(self):
        print( 'FT:', self.ft)
        print( 'Ebc:', self.ebc)
        for st in self.st_lst:
            st.show_st()

            
#BETA-BINOMIAL (BB) MODEL FOR CELL DIVISION,
#WHERE A FAMILY DEVELOPS FROM A 1 CELL IN GENERATION 0 ITERATIVELY AS FOLLOWS:
#THE NUMBER OF CELLS THAT DIVIDE FROM GENERATION i TO i+1 FOLLOWS A BETA-BINOMIAL DISTRIBUTION
#PARAMETRISED SO THAT ONE CELL DIVIDES WITH PROBABILITY p, AND THE CELLS IN THE SAME GENERATION
#DECIDE TO DIVIDE WITH (INTRACLASS) CORRELATION rho.
#IN ACCORDANCE WITH EXPERIMENTAL DATA, THE CELLS FROM FULL FAMILY VECTOR GROWN UNDER THE BB MODEL
#ARE THEN SAMPLED (INDEPENDENTLY AND WITH EQUAL DISTRIBUTION) WITH PROBABILITY r.

#GIVEN A FULL FAMILY VECTOR m, RETURN THE NUMBER OF CELLS THAT WERE EVER BORN IN EACH GENERATION
def cells_arrived_in_gen(m):
    nn = np.zeros(len(m))
    nn[0] = 1.
    for k in range(1, len(m)):
        nn[k] = 2.*(nn[k-1] - m[k-1])
    return nn

#PROBABILITY THAT A BETA-BINOMIAL DISTRIBUTION, WITH PARAMETERS n, a, b, EQUALS k
def bb_dist(k, n, a, b):
    if a == 0.:
        if k==0.:
            return 1.
        else:
            return 0.
    elif b == 0.:
        if k == n:
            return 1.
        else:
            return 0.
    else:
        return spsp.binom(n,k) * spsp.beta(k+a,n-k+b) / spsp.beta(a,b)
    
#PROBABILITY THAT A BINOMIAL DISTRIBUTION, WITH PARAMETERS n, R, EQUALS k    
def bino_dist(k, n, r):
    if r == 0.:
        if k==0.:
            return 1.
        else:
            return 0.
    elif r == 1.:
        if k == n:
            return 1.
        else:
            return 0.
    else:
        return spsp.binom(n, k) * r**(k) * (1.-r)**(n-k)
    
#PROBEBILITY OF OBSERVING THE FULL FAMILY VECTOR m UNDER THE BB MODEL WITH PARAMETERS p, rho
def pmf_m(m, p, rho):
    nn = cells_arrived_in_gen(m)
    if rho==0.0:
        res = np.prod([bino_dist(nn[i]-m[i], nn[i], p[i]) for i in range(len(m)-1)])
        return res
    elif rho==1.:
        m_v = [(m[dd] / (2.**dd)) == 1. for dd in range(len(m))]
        if sum(m_v) == 1.:
            n_dd = next(dd for dd in range(len(m)) if (m[dd]/(2.**dd))==1.)
            res = np.prod([p[i] for i in range(n_dd)])*(1.-p[n_dd])
            return res
        else:
            return 0.
    else:    
        alpha = np.multiply(p, (1.-rho)/rho)
        beta = np.multiply(np.subtract(1., p), (1.-rho)/rho)    
        res = np.prod([bb_dist(m[i], nn[i], beta[i], alpha[i])
                       for i in range(len(alpha)-1)])
        return res

#PROBABILITY OF OBSERVING THE SAMPLED FAMILY VECTOR v,
#SAMPLING EACH CELL FROM THE FULL FAMILY VECTOR m WITH SUCCESS RATE r
def pmf_v_given_m(v, m, r):
    return np.prod([bino_dist(v[i], m[i], r)
                    for i in range(len(v))]) / (1.-((1.-r)**sum(m)))

#PROBABILITY OF OBSERVING FRAY/RANGE EQUAL TO k UNDER THE BB MODEL rho, p,
#GENERATING FULL FAMILY VECTORS IN Ft_lst, THEN SAMPLED WITH SUCCESS RATE r 
def fray_dist(k, rho, p, r, Ft_lst):
    out = 0.
    for Ft in Ft_lst:
        out += sum([pmf_v_given_m(St.st, Ft.ft, r) for St in Ft.st_lst
                    if St.range==k]) * pmf_m(Ft.ft, p, rho)
    return out

#NEGATIVE LOG-LIKELIHOOD OF FRAY/RANGE DATA
def log_like_data(rho, p, r, Ft_lst, fray_freq):
    return -sum([fray_freq[k]*np.log(fray_dist(k, rho, p, r, Ft_lst))
                 for k in range(len(fray_freq))])

#BELOW ARE THE FUNCTIONS TO CALCLUATE THE DISTRIBUTION AND CONFIDENCE INTERVALS
#OF DATA IN A GIVEN SUPPORT (SIMILARLY AS IN NOTEBOOK 2)

#TRANSFORM THE PANDAS DATAFRAME OF A FAMILY INTO A FAMILY VECTOR
def df_fam_to_vec(dff, max_g):
    res = np.zeros(max_g+1)
    gens, counts = np.unique(dff.Generation, return_counts=True)
    res[gens] = counts
    return res

def epmf_on_support(data, support, bool_norm=True):
    classes, counts = np.unique(data, return_counts=True)
    counts_sum = counts.sum()
    if counts_sum == 0:
        return np.zeros(len(support))
    elif bool_norm:
        return np.hstack([counts[classes==cl] if any(classes==cl) else np.zeros(1) for cl in support]
                        )/counts_sum
    else:
        return np.hstack([counts[classes==cl] if any(classes==cl) else np.zeros(1) for cl in support])
    
def stat_bootCI(func, args, data, n_iter=250000, bool_trim01=True):
    stat = func(data, *args)
    boot_stat = np.array([func(data[boot_index], *args)
                          for boot_index in np.random.choice(len(data), size=(n_iter,len(data)))])
    boot_per = np.percentile(boot_stat, q=[97.5, 2.5], axis=0)
    boot_ci = 2.*stat - boot_per
    if bool_trim01:
        boot_ci[0,boot_ci[0,:]<0.] = np.zeros((boot_ci[0,:]<0.).sum())
        boot_ci[1,boot_ci[1,:]>1.] = np.ones((boot_ci[1,:]>1.).sum())
    return stat, boot_ci

In [11]:
###FIG 2F
df = df[df.Culture_time=='48h']#Use 48h data only
boot_iter = 250000
for n_oc,oc in enumerate(or_cells):
    for n_cnd,cnd in enumerate(conds):
        #if n_oc != 1 or n_cnd != 0:
        #    continue
            
        df_temp = df[(df.Original_cell==oc)&(df.Culture_condition==cnd)]
        
        #SET MAX GENERATION AND ALL SAMPLED FAMILY VECTORS FROM THE DATA 
        max_g = df_temp.Generation.max()
        uni_fams = np.unique(df_temp.Family)
        print('\n', oc, cnd, len(uni_fams))
        fam_vec = np.array([df_fam_to_vec(df_temp[df_temp.Family==fam], max_g)
                            for fam in uni_fams])
        
        #ESTIMATE p FOR THE BB MODEL, THE PROBABILITY FOR A CELL TO DIVIDE TO NEXT GENERATION 
        cc_pop = fam_vec.sum(axis=0)*(2.** -np.arange(max_g+1))
        p_div = np.array([cc_pop[i+1:].sum()/cc_pop[i:].sum() for i in np.arange(len(cc_pop)-1)] + [0.])
        
        #CALCULATE THE OBSERVED FRAY/RANGE, AND ITS DISTRIBUTION, FROM THE DATA
        fray_data = np.array([df_temp[df_temp.Family==fam].Generation.max()
                              - df_temp[df_temp.Family==fam].Generation.min()
                              for fam in uni_fams])
        fray_support = np.arange(max_g)
        fray_freq = epmf_on_support(fray_data, fray_support, bool_norm=False)
        
        #COMMENT/UNCOMMENT TO RECALCULATE THE OPTIMISATION STEP
        fray_epmf, boot_ci = stat_bootCI(func=epmf_on_support, args=(fray_support,), data=fray_data)

        #Generate all full amily vectors up to max observed depth d
        #start = time.process_time()
        samp_vec = np.array([fam for fam in fam_generator(max_g) if (fam * 2.**-np.arange(len(fam))).sum()<=1.
                                                    and (fam * 2.**-np.arange(len(fam))).sum()>=0.])
        full_vec = np.array([fam for fam in samp_vec if (fam * 2.**-np.arange(len(fam))).sum()==1.])
        Ft_lst = [Full_tree(fam) for fam in full_vec]
        #end = time.process_time()
        #print('Full trees generation time:', end-start)
        
        #MLE rho estimation
        #start = time.process_time()
        solution = spo.minimize_scalar(log_like_data, bounds=(0., 1.),
                            args=(p_div, r_sample, Ft_lst, fray_freq),
                            method='bounded', options={'disp': True})
        est_rho = solution.x
        #end = time.process_time()
        #print('MLE rho estimation time:', end-start)
        
        #BOOTSTRAP RHO
        boot_n_rho = 100
        boot_rho = []
        for boot_data in np.random.choice(fray_data, size=(boot_n_rho,len(fray_data))):
            boot_freq = epmf_on_support(boot_data, fray_support, bool_norm=False)
            boot_solution = spo.minimize_scalar(log_like_data, bounds=(0., 1.),
                                    args=(p_div, r_sample, Ft_lst, boot_freq),
                                    method='bounded', options={'disp': False})
            boot_rho.append(boot_solution.x)
    
        #Fray distribution under the BB model for other values of rho
        rho_lst = np.array([est_rho, 0.9, 0.5, 0.0])
        fray_pmf = {str(rho):[fray_dist(gen, rho, p_div, r_sample, Ft_lst)
                              for gen in fray_support] for rho in rho_lst}
        
        #LIKELIHOOD-RATIO TEST (EXTRA)
        neg_log_like_H0 = log_like_data(0., p_div, r_sample, Ft_lst, fray_freq)
        neg_log_like_H1 = log_like_data(est_rho, p_div, r_sample, Ft_lst, fray_freq)
        like_ratio_stat = 2. * (neg_log_like_H0 - neg_log_like_H1)
        pval = sps.chi2.sf(like_ratio_stat, df=1)
        print('LR test pval=', pval)
        
        #SAVE VARIABLES THAT TAKE A LONG TIME TO CALCULATE
        with open('./pickled_data/Fray_ci_'+oc+cnd, 'wb') as fp:
            pickle.dump((fray_epmf,boot_ci,est_rho,boot_rho,fray_pmf,pval), fp)
        #COMMENT/UNCOMMENT ENDS HERE
        
        with open('./pickled_data/Fray_ci_'+oc+cnd, 'rb') as fp:
            fray_epmf,boot_ci,est_rho,boot_rho,fray_pmf,pval = pickle.load(fp)

        #PRINT RHO CI
        boot_per_rho = np.percentile(boot_rho, q=[97.5, 2.5], axis=0)
        boot_ci_rho = 2.*est_rho - boot_per_rho
        print('Best-fit rho: {} in 95% CI ({},{})'.format(round(est_rho,3),round(boot_ci_rho[0],3),round(boot_ci_rho[1],3)))            
        
        #PLOTTING CODE
        rho_lst = np.array([est_rho, 0.9, 0.5, 0.0])###THIS IS ADDED FOR WHEN THE PART ABOVE IS COMMENTED
        width = 0.5/len(rho_lst)
        t_cols = sns.color_palette('Blues', n_colors=6)[::-1]
        color_dct = {str(est_rho):t_cols[0], '0.9':t_cols[1],
                     '0.5':t_cols[2], '0.0':t_cols[3]}
        shift_dct = {str(rho_lst[k]):-0.30 + (k+1)*0.125 for k in range(len(rho_lst))}    
        sns.set_style("ticks")
        fig, ax = plt.subplots()
        ax.bar(x=fray_support-0.30, height=fray_epmf, width=width, align='edge',
           color='Red', label='Data')
        for rho in rho_lst:
            if rho == est_rho:
                ax.bar(x=fray_support+shift_dct[str(rho)], height=fray_pmf[str(rho)],
                       width=width, align='edge', color=color_dct[str(rho)], hatch='///',
                       label=r'$\rho = $'+str(round(rho, 3))+' (Best fit)')
            else:
                ax.bar(x=fray_support+shift_dct[str(rho)], height=fray_pmf[str(rho)],
                       width=width, align='edge', color=color_dct[str(rho)],
                       label=r'$\rho = $'+str(round(rho, 3)))     
        xx_fray = np.add(np.arange(len(fray_epmf)), (width/2.)-0.30)
        for k in range(len(fray_epmf)):
            ax.plot([xx_fray[k]]*2, [boot_ci[0,k], boot_ci[1,k]],
                     color='Black', ls='-', lw=1., mfc='Black', mec='Black',
                     marker='_', mew=1, ms=5.)
        sns.despine(fig=fig)
        ax.set_xlabel('Range', fontsize='xx-large')
        ax.set_ylabel('% of clones', fontsize='xx-large')
        ax.set_xticks(np.arange(max_g))#len(fray_epmf)))
        ax.set_xticklabels(np.arange(max_g), fontsize='x-large')# np.arange(len(fray_epmf))
        ax.set_yticks(np.arange(0., 1.1, 0.25))
        ax.set_yticklabels(('0', '25', '50', '75', '100'), fontsize='x-large')
        if oc == 'SLAM-HSC' and cnd == 'P':
            ax.set_xlim(-0.5, 2.5)
        elif oc == 'SLAM-HSC' and cnd == 'P+ILs':
            ax.set_xlim(-0.5, 3.5)
        elif oc == 'ST-HSC' and cnd == 'P':
            ax.set_xlim(-0.5, 3.5)
        elif oc == 'ST-HSC' and cnd == 'P+ILs':
            ax.set_xlim(-0.5, 4.5)
        elif oc == 'MPP' and cnd == 'P':
            ax.set_xlim(-0.5, 2.5)
        elif oc == 'MPP' and cnd == 'P+ILs':
            ax.set_xlim(-0.5, 4.5)
        #ax.set_xlim(-0.5, fray_data.max()+0.5)
        ax.set_ylim(bottom=-0.0, top=1.0)
        ax.legend(fontsize='x-large', loc='upper left', ncol=1, bbox_to_anchor= (1.05, 1.))
        ax.set_title('Generation range 48h '+oc+' '+cnd, fontdict = {'fontsize':'xx-large'})
        fig.savefig(path_plot+'/Fray fit 48h '+oc+' '+cnd+'.pdf', bbox_inches='tight')


 ST-HSC P 77
Full trees generation time: 0.017768122999996194

Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )
MLE rho estimation time: 0.4749418570000046
LR test pval= 9.876525720395996e-12
Best-fit rho: {} in ({},{})


AttributeError: 'NoneType' object has no attribute 'format'

Best-fit rho: 0.799 in 95% CI (0.668,0.947)
